In [10]:
import pandas as pd
import os
import sys

module_path = os.path.abspath(os.path.join('../../py-conjugated/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import network_utils as nuts

In [3]:
#Take in data as a dataframe for easy preprocessing
total_df = pd.read_excel('/Users/wesleytatum/Desktop/py-conjugated/data/OPV_total_df.xlsx')
print (total_df.shape)
total_df.head()

(36, 60)


,Unnamed: 0,Anneal_time,Anneal_temp,Substrate,Device,p1_fraction,p2_fraction,p3_fraction,p1_area_avg,p1_area_stdev,...,p3_Ecc_avg,p3_Ecc_stdev,p3_Orient_avg,p3_Orient_stdev,p3_Perim_avg,p3_Perim_stdev,PCE,VocL,Jsc,FF
0,0,15,100,4,2,0.377864,0.397964,0.224172,138.756250,515.165733,...,0.806425,0.141707,-0.028958,1.145651,27.843437,33.295642,2.311387,0.577792,6.963861,57.44484
1,1,15,100,4,4,0.350601,0.368873,0.280526,78.978947,180.249005,...,0.817408,0.147199,0.053783,1.171795,42.334954,73.576430,1.763230,0.522596,7.047848,47.87256
2,2,15,100,4,7,0.290203,0.329655,0.380142,122.638743,246.651556,...,0.840800,0.129785,-0.158463,1.160457,42.913004,88.566792,2.394418,0.575790,6.939877,59.92170
3,3,30,100,7,2,0.255200,0.437277,0.307523,151.263158,607.110300,...,0.836830,0.122449,-0.486800,0.858037,34.541492,46.688722,2.628197,0.569525,7.301575,63.20169
4,4,30,100,7,4,0.363801,0.396645,0.239554,151.330935,441.730661,...,0.817914,0.131814,-0.111498,1.075222,29.762502,28.652323,2.710602,0.568631,7.328048,65.04993


In [9]:
Y = total_df[['PCE', 'VocL', 'Jsc', 'FF']] #target features to be predicted
X = total_df.drop(['Unnamed: 0','PCE', 'VocL', 'Jsc', 'FF'], axis = 1) #input features used to make prediction

In [8]:
x,y = X.shape

normed_X = pd.DataFrame()

X_norm_key = {}

for colname, coldata in X.iteritems():
    max_val = coldata.max()
    min_val = coldata.min()
    
    X_norm_key[colname] = [min_val, max_val]
    
    normed_col = (coldata - min_val) / (max_val - min_val)
    normed_X[colname] = normed_col
    
print (X_norm_key)
normed_X.head()    

{'Anneal_time': [0, 30], 'Anneal_temp': [0, 175], 'Substrate': [1, 8], 'Device': [1, 8], 'p1_fraction': [0.2485966810230451, 0.5888691307845378], 'p2_fraction': [0.1251534781320124, 0.4930661832222824], 'p3_fraction': [0.2165579794101096, 0.3968688126390562], 'p1_area_avg': [29.50933786078098, 175.6946564885496], 'p1_area_stdev': [46.38652305896206, 1170.579809327209], 'p1_filled_area_avg': [30.57555178268251, 194.4179104477612], 'p1_filled_area_stdev': [49.58336428191448, 1345.46478563035], 'p1_extent_avg': [0.493128951863037, 0.6241187149650081], 'p1_extent_stdev': [0.1404857288988411, 0.2166691369983912], 'p1_MajorAL_avg': [9.177927908753725, 20.35929048563928], 'p1_MajorAL_stdev': [6.854708787015648, 34.31649493070623], 'p1_MinorAL_avg': [4.682076682149734, 9.22944322343303], 'p1_MinorAL_stdev': [3.604720637690147, 13.47803449164003], 'p1_Ecc_avg': [0.794558920796938, 0.911964589525746], 'p1_Ecc_stdev': [0.09735926972520725, 0.1604045368548698], 'p1_Orient_avg': [-0.451050996652246

,Anneal_time,Anneal_temp,Substrate,Device,p1_fraction,p2_fraction,p3_fraction,p1_area_avg,p1_area_stdev,p1_filled_area_avg,...,p3_MajorAL_avg,p3_MajorAL_stdev,p3_MinorAL_avg,p3_MinorAL_stdev,p3_Ecc_avg,p3_Ecc_stdev,p3_Orient_avg,p3_Orient_stdev,p3_Perim_avg,p3_Perim_stdev
0,0.5,0.571429,0.428571,0.142857,0.379893,0.741510,0.042227,0.747318,0.416992,0.747056,...,0.369497,0.224733,0.470751,0.459038,0.116318,0.683351,0.587973,0.725402,0.385802,0.214615
1,0.5,0.571429,0.428571,0.428571,0.299771,0.662438,0.354767,0.338403,0.119074,0.308619,...,0.662014,0.720357,0.679762,0.839553,0.189330,0.745316,0.668887,0.752081,0.756401,0.786992
2,0.5,0.571429,0.428571,0.857143,0.122273,0.555842,0.907236,0.637064,0.178141,0.581681,...,0.683919,0.649905,0.674337,1.000000,0.344826,0.548846,0.461326,0.740511,0.771184,1.000000
3,1.0,0.571429,0.857143,0.142857,0.019406,0.848362,0.504491,0.832873,0.498779,0.868876,...,0.638331,0.686998,0.500720,0.480947,0.318441,0.466093,0.140235,0.431908,0.557095,0.404926
4,1.0,0.571429,0.857143,0.428571,0.338566,0.737923,0.127534,0.833337,0.351669,0.782907,...,0.514102,0.315159,0.493588,0.259797,0.192689,0.571745,0.507254,0.653533,0.434879,0.148635


In [23]:
def df_normalizer(dataframe):
    
    df = dataframe
    
    normed_df = pd.DataFrame()

    df_norm_key = {}

    for colname, coldata in df.iteritems():
        max_val = coldata.max()
        min_val = coldata.min()

        df_norm_key[colname] = [min_val, max_val]

        normed_col = (coldata - min_val) / (max_val - min_val)
        normed_df[colname] = normed_col
        
    return normed_df, df_norm_key 

In [11]:
df, key = nuts.df_Gaussian_normalize(X)

print (key)
df.head()

{'Anneal_time': [13.61111111111111, 10.324112629533909], 'Anneal_temp': [118.75, 50.488683314750276], 'Substrate': [3.6944444444444446, 2.4121057124956127], 'Device': [3.7222222222222223, 2.159511996607582], 'p1_fraction': [0.36631622783672274, 0.05971740099044536], 'p2_fraction': [0.3595097354140405, 0.05762926668256747], 'p3_fraction': [0.2741740367492367, 0.04207743528741292], 'p1_area_avg': [110.98546125778302, 34.99532057854271], 'p1_area_stdev': [313.6695279317548, 200.61976318631577], 'p1_filled_area_avg': [117.35416719532587, 39.00895186549878], 'p1_filled_area_stdev': [344.54408257372955, 232.78578066293602], 'p1_extent_avg': [0.5447321053007544, 0.027155307833556148], 'p1_extent_stdev': [0.17369036107462793, 0.016083643655062017], 'p1_MajorAL_avg': [15.761510011231175, 2.273255616603537], 'p1_MajorAL_stdev': [19.617391447251226, 5.376007907219412], 'p1_MinorAL_avg': [7.268047204332914, 1.1567286311087144], 'p1_MinorAL_stdev': [9.075122137076061, 2.1962890085084608], 'p1_Ecc_a

,Anneal_time,Anneal_temp,Substrate,Device,p1_fraction,p2_fraction,p3_fraction,p1_area_avg,p1_area_stdev,p1_filled_area_avg,...,p3_MajorAL_avg,p3_MajorAL_stdev,p3_MinorAL_avg,p3_MinorAL_stdev,p3_Ecc_avg,p3_Ecc_stdev,p3_Orient_avg,p3_Orient_stdev,p3_Perim_avg,p3_Perim_stdev
0,0.134529,-0.37137,0.126676,-0.797505,0.193368,0.667277,-1.188334,0.793557,1.004369,0.913145,...,-0.673634,-0.414521,0.033414,0.356250,-0.697599,0.527701,-0.456310,0.670559,-0.326996,-0.131734
1,0.134529,-0.37137,0.126676,0.128630,-0.263166,0.162474,0.150968,-0.914594,-0.665042,-0.928346,...,0.709939,1.569640,1.104187,2.112554,-0.356857,0.880726,-0.077524,0.824481,1.508956,2.345756
2,0.134529,-0.37137,0.126676,1.517833,-1.274562,-0.518051,2.518415,0.332995,-0.334055,0.218550,...,0.813544,1.287595,1.076398,2.853113,0.368827,-0.238597,-1.049187,0.757731,1.582190,3.267746
3,1.587438,-0.37137,1.370402,-0.797505,-1.860699,1.349434,0.792568,1.150945,1.462671,1.424802,...,0.597918,1.436092,0.186949,0.457373,0.245691,-0.710058,-2.552315,-1.022735,0.521591,0.692014
4,1.587438,-0.37137,1.370402,0.128630,-0.042111,0.644378,-0.822775,1.152882,0.638328,1.063724,...,0.010328,-0.052514,0.150410,-0.563367,-0.341182,-0.108138,-0.834180,0.255918,-0.083867,-0.417324


In [27]:
def df_denormalize(normed_df, norm_key):
    
    denormed_df = pd.DataFrame()
    
    for colname, coldata in normed_df.iteritems():
        mn = norm_key[colname][0]
        mx = norm_key[colname][1]
        
        denormed_col = (coldata * (mx - mn)) + mn
        
        denormed_df[colname] = denormed_col
        
    return denormed_df

In [28]:
test = df_denormalize(df, key)

test.head()

,Anneal_time,Anneal_temp,MajorAL_avg,MajorAL_stdev,MinorAL_avg,MinorAL_stdev,Ecc_avg,Ecc_stdev,Orient_avg,Orient_stdev,Perim_avg,Perim_stdev,GMM_label
0,15.0,100.0,17.194652,29.172066,8.554721,14.246681,0.820518,0.142370,-0.050137,1.067225,97.003245,352.677591,1.0
1,15.0,100.0,13.709728,14.635628,6.938374,7.985216,0.824877,0.138410,-0.205623,1.075388,48.559552,98.492589,1.0
2,15.0,100.0,18.211098,21.013541,8.857453,10.356933,0.826205,0.148190,-0.414390,1.047842,75.736366,151.456627,1.0
3,30.0,100.0,19.640309,46.935628,9.781374,26.994708,0.857443,0.105577,-0.331902,0.940640,236.482362,1306.085256,1.0
4,30.0,100.0,20.647291,35.791550,8.412561,11.216702,0.833485,0.136928,-0.454930,0.862964,96.093361,315.148913,1.0
